In [ ]:
#Depêndencias

!pip install fastapi uvicorn python-multipart pyngrok google-generativeai pymongo PyMuPDF
!pip install --upgrade pymongo


In [ ]:
import nest_asyncio
import uvicorn
import threading
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import google.generativeai as genai
import json
import fitz
from pymongo import MongoClient
from bson import ObjectId
import logging
from pydantic import BaseModel, Field
from starlette.requests import Request

nest_asyncio.apply()


app = FastAPI()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

client = MongoClient("") #String de conexão MongoDB Atlas
db = client[""] # Nome do banco de dados
collection = db["pdf_texts"]
logger.info("Conexão com o MongoDB foi estabelecida com sucesso.")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


genai.configure(api_key="") # Gemini API KEY
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config={
        'temperature': 0.5,
        'top_k': 0,
        'top_p': 0.95,
        'max_output_tokens': 1000
    },
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }
)

class AskRequest(BaseModel):
    pdf_id: str = Field(..., example="60f6e8b3a5c28e0c401c81d2")
    question: str = Field(..., example="Qual é a empresa contratante?")

@app.post("/upload-pdf/")
async def upload_pdf(file: UploadFile = File(...)):

    logger.info(f"Recebendo arquivo: {file.filename} com tipo: {file.content_type}")

    if file.content_type != "application/pdf":
        logger.warning(f"Arquivo enviado não é um PDF: {file.content_type}")
        raise HTTPException(status_code=400, detail="O arquivo enviado não é um PDF.")

    pdf_content = await file.read()
    logger.info(f"Conteúdo do PDF lido com sucesso. Tamanho: {len(pdf_content)} bytes")

    try:
        document = fitz.open(stream=pdf_content, filetype="pdf")
    except Exception as e:
        logger.error(f"Erro ao abrir o PDF: {e}")
        raise HTTPException(status_code=400, detail="Erro ao processar o PDF.")

    text = ""
    for page_num, page in enumerate(document):
        try:
            page_text = page.get_text()
            logger.info(f"Texto extraído da página {page_num}: {page_text[:100]}...")
            text += page_text
        except Exception as e:
            logger.error(f"Erro ao extrair texto da página {page_num}: {e}")
            continue

    document.close()

    if text:
        pdf_entry = {"text": text}
        try:
            result = collection.insert_one(pdf_entry)
            logger.info(f"Texto salvo no MongoDB com sucesso. ID do documento: {result.inserted_id}")
            return {"text": text, "pdf_id": str(result.inserted_id)}
        except Exception as e:
            logger.error(f"Erro ao salvar no MongoDB: {e}")
            raise HTTPException(status_code=500, detail="Erro ao salvar o texto no banco de dados.")
    else:
        logger.warning("Nenhum texto foi extraído do PDF.")
        raise HTTPException(status_code=400, detail="Nenhum texto foi extraído do PDF.")

async def ask_gemini(text, question):
    logger.info("Iniciando interação com a IA.")
    try:
        logger.info("Configurando histórico da conversa.")
        gemini = model.start_chat(history=[
            {
                'role': 'user',
                'parts': [
                    {'text': json.dumps({"pdf_text": text})},
                    {
                        'text': (
                            "Você é um especialista em informações extraídas de documentos. "
                            "Por favor, responda a pergunta a seguir **somente** no formato JSON sem qualquer outra formatação ou texto adicional. "
                            "Use exatamente as chaves \"resposta\", \"explicação\", e \"contexto\". "
                            "Por exemplo: {\"resposta\": \"sua resposta aqui\", \"explicação\": \"informações relevantes\", \"contexto\": \"contexto aqui\"}."
                        )
                    },
                    {'text': question}
                ]
            }
        ])

        logger.info("Enviando mensagem para a IA.")
        gemini.send_message(question)

        logger.info("Obtendo resposta da IA.")
        resposta = gemini.last.text.strip()
        logger.info(f"Resposta recebida: {resposta}")

        try:
            if resposta.startswith('```json'):
                logger.info("Resposta inicia com ```json, removendo formatação.")
                resposta = resposta[7:-3].strip()
            logger.info("Tentando converter a resposta para JSON.")
            resposta_json = json.loads(resposta)


            logger.info("Resposta JSON convertida com sucesso.")
            return resposta_json

        except json.JSONDecodeError as jde:
            logger.warning(f"Resposta da IA não estava no formato JSON: {jde}")
            return {
                "resposta": "Desculpe, não consegui entender a resposta da IA.",
                "explicação": "",
                "contexto": ""
            }
    except Exception as e:
        logger.error(f"Erro durante a interação com a IA: {e}")
        return {
            "resposta": "Desculpe, ocorreu um erro ao processar sua pergunta.",
            "explicação": str(e),
            "contexto": ""
        }

@app.post("/ask/")
async def ask_about_pdf(request: AskRequest):
    logger.info(f"Recebendo requisição para /ask/ com pdf_id: {request.pdf_id} e question: {request.question}")

    try:
        pdf_entry = collection.find_one({"_id": ObjectId(request.pdf_id)})
    except Exception as e:
        logger.error(f"Erro ao buscar PDF no MongoDB: {e}")
        raise HTTPException(status_code=500, detail="Erro ao buscar o PDF no banco de dados.")

    if not pdf_entry:
        logger.warning(f"PDF não encontrado com ID: {request.pdf_id}")
        raise HTTPException(status_code=404, detail="PDF não encontrado.")

    pdf_text = pdf_entry.get("text", "")
    if not pdf_text:
        logger.warning(f"Nenhum texto encontrado para o PDF ID: {request.pdf_id}")
        raise HTTPException(status_code=400, detail="Nenhum texto encontrado para este PDF.")

    logger.info(f"Fazendo pergunta para o PDF ID: {request.pdf_id}")

    response = await ask_gemini(pdf_text, request.question)

    logger.info("Retornando resposta para o cliente.")
    return response

@app.post("/test/")
async def test_endpoint(request: AskRequest):
    return {"resposta": "Teste", "explicação": "Este é um teste.", "contexto": "Teste de endpoint."}

@app.middleware("http")
async def log_requests(request: Request, call_next):
    try:
        body = await request.body()
        logger.info(f"Recebendo requisição em {request.url.path} com corpo: {body}")
    except Exception as e:
        logger.error(f"Erro ao ler o corpo da requisição: {e}")
    response = await call_next(request)
    return response


class Interaction(BaseModel):
    pdf_id: str = Field(..., example="60f6e8b3a5c28e0c401c81d2")
    question: str = Field(..., example="Qual é a empresa contratante?")
    response: dict = Field(..., example={"resposta": "empresa X", "explicação": "informações relevantes", "contexto": "contexto aqui"})

@app.post("/store-interaction/")
async def store_interaction(interaction: Interaction):
    interaction_data = {
        "pdf_id": interaction.pdf_id,
        "question": interaction.question,
        "response": interaction.response
    }

    result = collection.insert_one(interaction_data)
    logger.info(f"Interação salva no MongoDB com sucesso. ID do documento: {result.inserted_id}")

    return {"message": "Interação armazenada com sucesso.", "interaction_id": str(result.inserted_id)}

In [ ]:
import os
from pyngrok import ngrok

# Adicionar o authtoken do ngrok
!ngrok config add-authtoken exemplo__seu_auth_token_aqui

# Criar um túnel HTTP para a porta 8000 e obter apenas a URL pública
tunnel = ngrok.connect(8000, "http")
public_url = tunnel.public_url
print("API URL:", public_url)

In [ ]:
import threading

# Permitir a execução de asyncio em um ambiente já ativo
nest_asyncio.apply()

# Função para rodar o servidor FastAPI
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Criar e iniciar uma nova thread para rodar o servidor
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()


In [ ]:
#Enviar PDF

import requests
from google.colab import files

def upload_pdf_and_get_id(public_url):
    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo foi enviado.")
        return None

    file_name = list(uploaded.keys())[0]
    print(f"Arquivo enviado: {file_name}")

    with open(file_name, 'rb') as f:
        files_payload = {'file': (file_name, f, 'application/pdf')}
        response = requests.post(f"{public_url}/upload-pdf/", files=files_payload)

    if response.status_code == 200:
        pdf_id = response.json().get("pdf_id")
        print("Upload bem-sucedido. ID do PDF:", pdf_id)
        return pdf_id
    else:
        print(f"Erro ao fazer upload: {response.status_code}")
        print(response.text)
        return None

pdf_id = upload_pdf_and_get_id(public_url)

In [ ]:
#Interação pelo Collab e já salvando as interações no db

def ask_question(public_url, pdf_id):
    if not pdf_id:
        print("Nenhum PDF_id disponível. Faça o upload de um PDF primeiro.")
        return

    while True:
        question = input("Digite a sua pergunta sobre o PDF (ou 'sair' para encerrar): ")
        if question.lower() == 'sair':
            break

        payload = {
            "pdf_id": pdf_id,
            "question": question
        }


        response = requests.post(f"{public_url}/ask/", json=payload)

        if response.status_code == 200:
            answer = response.json()
            print("Resposta da IA:")
            print(json.dumps(answer, indent=4, ensure_ascii=False))

            store_payload = {
                "pdf_id": pdf_id,
                "question": question,
                "response": answer
            }
            store_response = requests.post(f"{public_url}/store-interaction/", json=store_payload)

            if store_response.status_code == 200:
                print("Interação armazenada com sucesso.")
            else:
                print(f"Erro ao armazenar a interação: {store_response.status_code}")
                print(store_response.text)
        else:
            print(f"Erro ao fazer a pergunta: {response.status_code}")
            print(response.text)

if pdf_id:
    ask_question(public_url, pdf_id)